In [1]:
!pip install -qU langchain openai

# Quick Agents Intro

#### [LangChain Handbook](https://pinecone.io/learn/langchain/)

In this notebook we'll see a quick intro to LangChain *agents*. For a more thorough introduction, check out [this notebook](https://github.com/pinecone-io/examples/blob/master/generation/langchain/handbook/06-langchain-agents.ipynb).

To use agents we need three main components:

* **L**arge **L**anguage **M**odels (**LLMs**)
* **Tools**
* The **Agents** themselves

Let's get started by initializing a LLM:

In [2]:
from langchain import OpenAI

llm = OpenAI(
    openai_api_key="OPENAI_API_KEY",
    temperature=0,
    model_name="text-davinci-003"
)

Next, initialize a *calculator* tool using a `LLMMathChain`:

In [3]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain(llm=llm)

# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]

Now we use our `tools` (containing a single tool for now) to initialize an agent:

In [4]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
	agent="zero-shot-react-description",
	tools=tools,
	llm=llm,
	verbose=True,
	max_iterations=3
)

Let's try asking some questions:

In [5]:
zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
 I need to calculate this expression
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066

Thought: I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

In [6]:
(4.5*2.1)**2.2

139.94261298333066

Looks good, let's try something else:

In [7]:
zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")



> Entering new AgentExecutor chain...
 I need to figure out how many apples are in the boxes
Action: Calculator
Action Input: 8 * 2.5
Observation: Answer: 20.0

Thought: I need to add the apples Mary has to the apples in the boxes
Action: Calculator
Action Input: 4 + 20.0
Observation: Answer: 24.0

Thought: I now know the final answer
Final Answer: We have 24 apples.

> Finished chain.


{'input': 'if Mary has four apples and Giorgio brings two and a half apple boxes (apple box contains eight apples), how many apples do we have?',
 'output': 'We have 24 apples.'}

But what if we ask a non-math question? Something that a plain and simple LLM should be able to answer easily?

In [8]:
zero_shot_agent("what is the capital of Norway?")



> Entering new AgentExecutor chain...
 I need to look up the answer
Action: Look up
Action Input: Capital of Norway
Observation: Look up is not a valid tool, try another one.
 I need to find the answer using a tool
Action: Calculator
Action Input: N/A

ValueError: ignored

Unfortunately we end up hitting an error. That is because the agent contains only one option — that is to use the calculator tool.

If we'd like the agent to answer this "general knowledge" question. We can, we just need to link it to an LLM tool.

We initialize the tool like so:

In [9]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

# initialize the LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='use this tool for general purpose queries and logic'
)

Add the new tool to our `tools` list:

In [10]:
tools.append(llm_tool)

Now we reinitialize the agent with our two tools (`"Calculator"` and `"Language Model"`)

In [11]:
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

In [12]:
zero_shot_agent("what is the capital of Norway?")



> Entering new AgentExecutor chain...
 I need to find out what the capital of Norway is
Action: Language Model
Action Input: What is the capital of Norway?
Observation: 

The capital of Norway is Oslo.
Thought: I now know the final answer
Final Answer: The capital of Norway is Oslo.

> Finished chain.


{'input': 'what is the capital of Norway?',
 'output': 'The capital of Norway is Oslo.'}

In [13]:
zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
 I need to calculate this expression
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066

Thought: I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

---